  # Exploring Precipitation Patterns in UK Catchments







  ## Objectives



  By the end of this tutorial, you will:



  - Understand and calculate key precipitation indices



  - Visualize precipitation data using various techniques



  - Reinforce Python skills in data analysis and visualization.







  ## Prerequisites:



  - Basic Python understanding.



  - Familiarity with pandas and matplotlib.

  ## Dataset



  We will return to the CAMELS-GB dataset we used previously. Let's load data for one catchment:

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from sklearn.linear_model import TheilSenRegressor
from pathlib import Path

# Use upper case for constants
SHARED_DATADIR = Path(os.environ["SHARED_DATA_DIR"])
DATADIR = SHARED_DATADIR / '8344e4f3-d2ea-44f5-8afa-86d2987543a9' / 'data'

# Load the data
id = '97002'
data = pd.read_csv(os.path.join(DATADIR, 'timeseries', f'CAMELS_GB_hydromet_timeseries_{id}_19701001-20150930.csv'), parse_dates=[0])
data.head()



  It's a good idea to check how many NaN values we have in the precipitation data:

In [ ]:
# Check for missing values
print(data['precipitation'].isnull().sum())
# # Fill missing data or drop (depending on context)
# data.fillna(method='ffill', inplace=True)




  Let's start by plotting a scatter plot of the data:





In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(data['date'], data['precipitation'], color='b', alpha=0.7)
plt.gca().xaxis.set_major_locator(mdates.YearLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
plt.xticks(rotation=45)
plt.title('Daily precipitation')
plt.ylabel('Precipitation (mm)')
plt.tight_layout()
plt.show()




  This is a lot of data to try to interpret. One thing that may interest us is the annual maximum daily precipitation. We can find this by grouping our dataframe by year, and retrieving the maximum value:

In [ ]:
amax = data.groupby(data['date'].dt.year)[['precipitation']].max().reset_index()
amax.head()



In [ ]:
# Plotting
plt.figure(figsize=(10, 6))
plt.scatter(amax['date'], amax['precipitation'], color='b', alpha=0.7)
# plt.gca().xaxis.set_major_locator(mdates.YearLocator())
# plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
plt.xticks(rotation=45)
plt.title('Annual maximum daily precipitation')
plt.ylabel('Precipitation (mm)')
plt.tight_layout()
plt.show()



  It looks as though there might be a trend. How can we assess this quantitatively? One commonly used approach is to use the Theil-Sen estimator. This method provides a robust estimate of the trend slope and is less sensitive to outliers than some alternatives (e.g. linear regression).

In [ ]:
years = amax['date'].values.reshape(-1, 1)
precipitation = amax['precipitation'].values

model = TheilSenRegressor()
model.fit(years, precipitation)

trend_line = model.predict(years)

# Plotting
plt.figure(figsize=(10, 6))
plt.scatter(amax['date'], amax['precipitation'], color='b', alpha=0.7, label='Data')
plt.plot(years, trend_line, color='red', label='Trend Line (Theil-Sen)')
plt.xticks(rotation=45)
plt.title('Trend in Annual Max Daily Precipitation')
plt.ylabel('Precipitation (mm)')
plt.legend()
plt.tight_layout()
plt.show()



  How can we summarise the trend, and compare it with other sites? We can extract the slope as follows:

In [ ]:
slope = model.coef_[0]
print(f"Slope: {slope}")


 ## Taking it further

 Using the code above as a starting point, compute the slope of the trend for multiple basins.



 Have a look at Prosdocimi et al. ([NHESS](https://doi.org/10.5194/nhess-14-1125-2014), 2014) for inspiration.



 Think about the limitations of this analysis.

 - Where has the data come from?

 - Is a linear trend appropriate?

 - Is the annual maximum precipitation the best value to use? What else might be more appropriate for different applications?

 ## UK precipitation and the North Atlantic Oscillation

 In the lecture we spoke about the role of large-scale climate oscillations on precipitation patterns worldwide. In the UK, the North Atlantic Oscillation (NAO) exerts considerable influence on winter (December-February; DJF) precipitation.

 First of all we are going to download the NAO index from the internet. The following code downloads the data using Python's `reqeuests` library and converts the data to a Pandas dataframe:

In [ ]:
import requests 
from io import StringIO

response = requests.get('https://crudata.uea.ac.uk/cru/data/nao/nao.dat')
data = response.text

# Convert string data into a StringIO object
data_io = StringIO(data)

# Read the data into a Pandas DataFrame
# Assuming the first value in each row is the index and the rest are data columns
nao_data = pd.read_csv(data_io, sep='\\s+', header=None)

# Set column names if needed
nao_data.columns = ['year'] + [f'{i}' for i in range(1, 13)] + ['annual_mean']
nao_data = nao_data.drop('annual_mean', axis=1)
nao_data = nao_data.melt(id_vars=['year'], var_name='month', value_name='nao')
nao_data.loc[nao_data['nao'] == -99.99, 'nao'] = np.nan

# Convert the Year/month columns to a date, and make this the index
nao_data['date'] = pd.to_datetime(nao_data['year'].astype(str) + '-' + nao_data['month'], format='%Y-%m')
nao_data = nao_data.set_index('date')

print(nao_data.head())


 The resulting dataframe contains monthly NAO values from 1821 to the present year, with missing values represented as NaN. Now we want to compute the DJF seasonal average, discarding other months. We can do this as follows:

In [ ]:
# Increment the year in December by 1, so that season_year is the year of the end month (i.e. Feb)
nao_data['season_year'] = nao_data.index.year 
nao_data.loc[nao_data.index.month == 12, 'season_year'] += 1 

# Define a custom season assignment
season_mapping = {12: 'DJF', 1: 'DJF', 2: 'DJF'} 

# Add a 'season' column
nao_data['season'] = nao_data.index.month.map(season_mapping)

# Drop rows that don't belong to DJF:
nao_data = nao_data[nao_data['season'] == 'DJF']

# Finally, compute the seasonal averages:
nao_data = nao_data.groupby(['season_year', 'season'])[['nao']].mean().reset_index()
print(nao_data)


 Now we need to prepare our rainfall data. Lets create a function to compute the DJF mean precipitation for any given catchment, using the same aggregation method as we used for the NAO:

In [ ]:
def compute_djf_precip(x): 

    x['date'] = pd.to_datetime(x['date'])
    x = x.set_index('date')
    x['season_year'] = x.index.year 
    x.loc[x.index.month == 12, 'season_year'] += 1

    # Define a custom season assignment
    season_mapping = {12: 'DJF', 1: 'DJF', 2: 'DJF'} 

    # Add a 'season' column
    x['season'] = x.index.month.map(season_mapping)

    # Drop rows that don't belong to a season
    x = x[x['season'] == 'DJF']

    # Compute seasonal averages
    x = x.groupby(['season_year', 'season'])[['precipitation']].mean().reset_index()
    return x


 We are interested in the relationship between preciptiation and NAO. Let's compute the correlation coefficient (R)

In [ ]:
from scipy.stats import pearsonr, spearmanr
from tqdm import tqdm 

data = pd.read_csv(
    os.path.join(DATADIR, 'timeseries', f'CAMELS_GB_hydromet_timeseries_{id}_19701001-20150930.csv'), 
    parse_dates=[0])

data = compute_djf_precip(data)
data = data.merge(nao_data, how='left', on=['season_year', 'season'])
pearson_corr = pearsonr(data['precipitation'], data['nao'])
result = pd.DataFrame.from_dict({'id': [f'{id}'], 'pearsonr': pearson_corr.statistic, 'pvalue': pearson_corr.pvalue})


 As in the previous notebook, let's load the catchment metadata (here we load the land use attributes - but it could be anything because we just want to get all the catchment IDs)

In [ ]:
metadata_lu = pd.read_csv(os.path.join(DATADIR, f'CAMELS_GB_landcover_attributes.csv')) 
catchment_ids = metadata_lu['gauge_id'].to_list()
print(len(catchment_ids))


 OK, now we can loop through the catchments and compute the correlation of winter precipitation and the NAO index.

In [ ]:
raise NotImplementedError()


 Now we should have a dataframe with columns `id`, `spearmanr` and `pvalue`. We're interested in (i) the strength of the relationship in various catchments, and (ii) how the spatial pattern varies in space. How might we visualize this information? One nice approach would be to create a choropleth map. You can do this by joining the dataset you have just created with the catchment polygons in the CAMELS-GB dataset - `data/8344e4f3-d2ea-44f5-8afa-86d2987543a9/data/CAMELS_GB_catchment_boundaries.shp`.

In [ ]:
raise NotImplementedError()